In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local').config('spark.driver.memory', '8g').appName('Read_parquet').getOrCreate()



In [17]:
df = spark.read.parquet(
    '/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/item_visits/*202*/*.parquet',header=True)


In [30]:
df.createOrReplaceTempView('item_visits')

df2 = spark.sql("SELECT item_code, meta_date, SUM(unique_views) from item_visits where meta_date <= '2020-11-18' GROUP BY item_code, meta_date")

In [31]:
df2.show()

+------------------+----------+-----------------+
|         item_code| meta_date|sum(unique_views)|
+------------------+----------+-----------------+
|000000001000207165|2020-11-01|                1|
|000000001000100776|2020-11-01|                1|
|000000001000179062|2020-11-01|               57|
|000000001000032299|2020-11-01|                1|
|000000001000133364|2020-11-01|                2|
|000000001000135791|2020-11-01|                1|
|000000001000148080|2020-11-01|               58|
|000000001000050941|2020-11-01|                3|
|000000001000176451|2020-11-01|               16|
|000000001000150403|2020-11-01|               14|
|000000001000186188|2020-11-01|                1|
|000000001000183985|2020-11-01|                9|
|000000001000054589|2020-11-01|               14|
|000000001000159402|2020-11-01|                1|
|000000001000121796|2020-11-01|                1|
|000000001000116886|2020-11-01|                3|
|000000001000128571|2020-11-01|                1|


In [5]:
df_d_item = spark.read.parquet('/Users/gabriele.sabato/PycharmProjects/raw_data/redshift_tables/d_item/*')


In [6]:
df_d_item.createOrReplaceTempView('d_item_tmp')

In [7]:
df2 = spark.sql("SELECT * from model_data_table where update_date >= '2020-01-01' ORDER BY update_date ")

In [8]:
df4 = spark.sql ("SELECT item_code, item_main_category, item_sub_category_1, item_sub_category_2,item_parent_item_code from d_item_tmp")

In [9]:
df4.createOrReplaceTempView('d_item_filtered')
df2.createOrReplaceTempView('model_data_table_YTD')

In [10]:
list_of_top_seller_item_code = ['000000001000016021','000000001000016133','000000001000022708', '000000001000015958',
                       '000000001000016020','000000001000015959','000000001000016019','000000001000022573','000000001000015944',
                       '000000001000024281']

In [11]:
top_seller_list_string = """'000000001000016021','000000001000016133','000000001000022708', '000000001000015958',
                            '000000001000016020','000000001000015959','000000001000016019','000000001000022573','000000001000015944',
                            '000000001000024281'"""

In [58]:
sql_top_item_query_group_by = """
WITH tmp_tbl AS (
      SELECT *,
             CASE
                 WHEN lag(delivery_weeks, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      delivery_weeks
                     AND
                      lag(item_price, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      item_price
                     THEN NULL
                 ELSE RANK() OVER (PARTITION BY item_code ORDER BY update_date)
                 END AS ranking_col --filled with row number or delivery week if the previous one is part of the same group,
        FROM model_data_table_YTD
        WHERE item_code in ({lista})
  ),
       tmp_tbl2 AS (
           SELECT update_date,
                  item_code,
                  sales,
                  item_price,
                  delivery_weeks,
                  CASE
                      WHEN ranking_col IS NULL
                          THEN last(ranking_col, True) OVER (PARTITION BY item_code ORDER BY update_date ROWS BETWEEN UNBOUNDED PRECEDING and 1 PRECEDING)
                      ELSE ranking_col
                      END AS ranks
             FROM tmp_tbl
       )
SELECT CAST(MIN(update_date) as date)                     AS min_date,
       CAST (MAX(update_date) as date)                    AS max_date,
       item_code,
       item_price,
       delivery_weeks,
       avg(sales),
       CAST (datediff(MAX(update_date), MIN(update_date) ) + 1 as int) AS bin
  FROM tmp_tbl2
 GROUP BY ranks, delivery_weeks, item_price, item_code
 ORDER BY item_code, min_date ASC;
""".format(lista=top_seller_list_string)

In [59]:
print(sql_top_item_query_group_by)


WITH tmp_tbl AS (
      SELECT *,
             CASE
                 WHEN lag(delivery_weeks, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      delivery_weeks
                     AND
                      lag(item_price, 1) OVER (PARTITION BY item_code ORDER BY update_date ASC) =
                      item_price
                     THEN NULL
                 ELSE RANK() OVER (PARTITION BY item_code ORDER BY update_date)
                 END AS ranking_col --filled with row number or delivery week if the previous one is part of the same group,
        FROM model_data_table_YTD
        WHERE item_code in ('000000001000016021','000000001000016133','000000001000022708', '000000001000015958',
                            '000000001000016020','000000001000015959','000000001000016019','000000001000022573','000000001000015944',
                            '000000001000024281')
  ),
       tmp_tbl2 AS (
           SELECT update_date,
                  item_code

In [60]:
sql_top_item_df = spark.sql(sql_top_item_query_group_by)

In [61]:
sql_top_item_df.show(5000,False)

+----------+----------+------------------+------------+--------------+------------------+---+
|min_date  |max_date  |item_code         |item_price  |delivery_weeks|avg(sales)        |bin|
+----------+----------+------------------+------------+--------------+------------------+---+
|2020-01-01|2020-01-02|000000001000015944|64.98999999 |2             |8.5               |2  |
|2020-01-03|2020-01-03|000000001000015944|69.98999999 |2             |7.0               |1  |
|2020-01-04|2020-01-10|000000001000015944|69.98999999 |1             |7.142857142857143 |7  |
|2020-01-11|2020-01-22|000000001000015944|64.98999999 |1             |6.5               |12 |
|2020-01-23|2020-01-24|000000001000015944|64.98999999 |2             |7.0               |2  |
|2020-01-25|2020-01-29|000000001000015944|64.98999999 |1             |5.8               |5  |
|2020-01-30|2020-01-31|000000001000015944|64.98999999 |2             |3.0               |2  |
|2020-02-01|2020-02-09|000000001000015944|64.98999999 |1    

In [62]:
#CREATE A TABLE VIEW FOR Top10 sellers
sql_top_item_df.createOrReplaceTempView('top_sellers')


In [63]:
#JOIN THE TWO TABLEs
sql_join_query = " SELECT top.*, di.item_parent_item_code, di.item_main_category, di.item_sub_category_1, di.item_sub_category_2" \
                 "        from top_sellers as top " \
                 "INNER JOIN d_item_filtered as di on di.item_code = top.item_code "\
                 "ORDER BY top.min_date ASC"

In [64]:
print(sql_join_query)


 SELECT top.*, di.item_parent_item_code, di.item_main_category, di.item_sub_category_1, di.item_sub_category_2        from top_sellers as top INNER JOIN d_item_filtered as di on di.item_code = top.item_code ORDER BY top.min_date ASC


In [65]:
final_top_seller_df = spark.sql(sql_join_query)

In [66]:
final_top_seller_df.show(3000,False)


+----------+----------+------------------+------------+--------------+------------------+---+---------------------+---------------------+-------------------+-------------------+
|min_date  |max_date  |item_code         |item_price  |delivery_weeks|avg(sales)        |bin|item_parent_item_code|item_main_category   |item_sub_category_1|item_sub_category_2|
+----------+----------+------------------+------------+--------------+------------------+---+---------------------+---------------------+-------------------+-------------------+
|2020-01-01|2020-01-05|000000001000016133|119.98999999|1             |10.6              |5  |000000008000001083   |ESSZIMMERSTÜHLE&BÄNKE|ESSZIMMERSTÜHLE    |KLASSISCH          |
|2020-01-01|2020-02-10|000000001000022708|99.98999999 |1             |3.1219512195121952|41 |000000008000017295   |MATRATZEN&LATTENROSTE|LATTENROSTE        |FEDERHOLZ          |
|2020-01-01|2020-01-01|000000001000016021|119.98999999|5             |23.0              |1  |00000000800000108

In [69]:
final_top_seller_df.describe(['bin']).show()
final_top_seller_df.withColumn('item_price',col('item_price').cast('double'))

+-------+------------------+
|summary|               bin|
+-------+------------------+
|  count|               200|
|   mean|             16.15|
| stddev|39.848425124772774|
|    min|                 1|
|    max|               275|
+-------+------------------+



In [70]:
pd_df_top_sellers = final_top_seller_df.toPandas()


In [71]:
import pickle
import pandas as pd
import numpy as np


In [72]:
pd_df_top_sellers.to_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top_seller_YTD_group_by.pickle')

In [73]:
pd_df_top_sellers['log_price'] = np.log(pd_df_top_sellers['item_price'])

In [74]:
pd_df_top_sellers['log_delivery_weeks'] = np.log(pd_df_top_sellers['delivery_weeks']+1)

In [75]:
pd_df_top_sellers['log_sales'] = np.log(pd_df_top_sellers['avg(sales)']+1)


In [76]:
pd_df_top_sellers.to_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top_seller_YTD_group_by.pickle')